<h1>Gradient Boosting</h1>

[Good explanation of how gradient boosting works step by step](https://www.youtube.com/watch?v=3CC4N4z3GJc&t=23s)

[Yandex ML Book Gradient Boosting](https://education.yandex.ru/handbook/ml/article/gradientnyj-busting)

[Scikit learn explanation of gradient boosting details](https://scikit-learn.org/stable/modules/ensemble.html#mathematical-formulation)

In [33]:
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_boston, make_classification
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, accuracy_score, roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier

from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

In [4]:
class DecisionTreeRegressor_:
    '''
    DecisionTreeRegressor class is used to permorm CART algorithm for Decision Tree Regressor.
    *****************************************************************************************
    Attributes:
    max_depth - int; adjusts the depth of the tree
    min_samples_split - int; set the minimum size to split
    *****************************************************************************************
    Methods:
    make_dataset - from X and y makes one matrix
    MSE - count the MSE
    test_split - split the dataset into two groups with the threshold 
    get_split - find the best split using the best criterion value
    to_terminal - make the final node
    split - build the tree, recursively
    fit - starts the tree building
    print_tree - print the tree
    predict_row - predict the row of data using the tree
    predict - predict the whole data
    '''
    def __init__(self, max_depth=np.infty, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        
    def make_dataset(self, X, y):
        return np.concatenate((X, y.reshape(-1, 1)), axis=1)
    
    def MSE(self, groups):
        n_instances = np.sum([len(group) for group in groups])
        criterion = 0
        for group in groups:
            size = len(group)
            if size == 0:
                continue
            score = np.mean(group[:, -1])
            criterion += np.sum((score - group[:, -1]) ** 2) # not weighted, just sum or residuals
        return criterion

    def test_split(self, index, value, dataset):
        left, right = list(), list()
        for row in dataset:
            if row[index] < value:
                left.append(row)
            else:
                right.append(row)
        return np.array(left), np.array(right)

    def get_split(self, dataset):
        b_index, b_value, b_score, b_groups, MSE_history = None, None, np.infty, None, []
        for index in range(len(dataset[0]) - 1):
            for row in dataset:
                groups = self.test_split(index, row[index], dataset)
                criterion = self.MSE(groups)
                MSE_history.append(criterion)
                if criterion <= b_score:
                    b_index, b_value, b_score, b_groups = index, row[index], criterion, groups
        if len(set(MSE_history)) == 1: # checking if there's no need to split dataset
            b_groups = self.test_split(0, np.min(dataset[:, 0]), dataset)
        try:
            b_value = (b_value + np.max(b_groups[0][:, b_index])) / 2
        except IndexError:
            pass
        return {'index':b_index, 'value':b_value, 'groups':b_groups}

    def to_terminal(self, group):
        outcomes = group[:, -1]
        return np.mean(outcomes)

    def split(self, node, depth):
        left, right = node['groups']
        del(node['groups'])
        if len(left) == 0 or len(right) == 0:
            node['left'] = node['right'] = self.to_terminal(np.array(left.tolist() + right.tolist()))
            return
        if depth >= self.max_depth:
            node['left'], node['right'] = self.to_terminal(left), self.to_terminal(right)
            return
        if len(left) < self.min_samples_split:
            node['left'] = self.to_terminal(left)
        else:
            node['left'] = self.get_split(left)
            self.split(node['left'], depth+1)
        if len(right) < self.min_samples_split:
            node['right'] = self.to_terminal(right)
        else:
            node['right'] = self.get_split(right)
            self.split(node['right'], depth+1)
            
    def fit(self, X, y):
        train = self.make_dataset(X, y)
        root = self.get_split(train)
        self.split(root, 1)
        self.node = root

    def print_tree(self, node, depth=0):
        if isinstance(node, dict):
            print('{0}[X[{1}] < {2}]'.format(depth * '>', node['index'], np.round(node['value'], 3)))
            self.print_tree(node['left'], depth+1)
            self.print_tree(node['right'], depth+1)
        else:
            print('{0}[{1}]'.format('   ' + depth * '>', node))
    
    def predict_row(self, node, row):
        if row[node['index']] < node['value']:
            if isinstance(node['left'], dict):
                return self.predict_row(node['left'], row)
            else:
                return node['left']
        else:
            if isinstance(node['right'], dict):
                return self.predict_row(node['right'], row)
            else:
                return node['right']
            
    def predict(self, X):
        predictions = np.array([])
        for row in X:
            predictions = np.append(predictions, self.predict_row(self.node, row))
        return predictions

In [5]:
class GradientBoostingRegressor:
    '''
    GradientBoostingRegressor class is used to permorm gragient boosting on trees.
    *****************************************************************************************
    Attributes:
    lr - learning rate
    n_iter - number of iterations
    max_depth - int; adjusts the depth of the tree
    min_samples_split - int; set the minimum size to split
    *****************************************************************************************
    Methods:
    fit - build the algorithm
    predict - predict the whole data
    '''
    def __init__(self, lr=0.1, n_iter=100, **model_kwargs):
        self.lr = lr
        self.n_iter = n_iter
        self.model = DecisionTreeRegressor(**model_kwargs)
        self.models = list()
        
        for i in range(n_iter):
            self.models.append(copy.deepcopy(self.model))
            
    def fit(self, x, y):
        approximation = np.zeros((x.shape[0])).reshape(-1)
        
        for model in self.models:
            grad = -(y.reshape(-1) - approximation)
            
            model.fit(x, grad) # fit model on residuals
            approximation -= self.lr * model.predict(x)
        return self
    
    def predict(self, x):
        approximation = np.zeros((x.shape[0])).reshape(-1)
        for model in self.models:
            approximation -= self.lr * model.predict(x)
        return approximation
    
    def __repr__(self):
        return f'GradientBoostingRegressor(lr={self.lr}, n_iter={self.n_iter})'

In [6]:
data = load_boston()
x = data['data']
y = data['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [7]:
boosting = GradientBoostingRegressor(lr=0.1, max_depth=3)
boosting.fit(x_train, y_train)

GradientBoostingRegressor(lr=0.1, n_iter=100)

In [8]:
print(f'{r2_score.__name__}: {r2_score(y_test, boosting.predict(x_test))}')

r2_score: 0.8826201874006322


In [9]:
my_tree = DecisionTreeRegressor_(max_depth=3)
my_tree.fit(x_train, y_train)

In [10]:
print(f'{r2_score.__name__}: {r2_score(y_test, my_tree.predict(x_test))}')

r2_score: 0.6860847825591382


In [11]:
skl_tree = DecisionTreeRegressor(max_depth=3)
skl_tree.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=3)

In [12]:
print(f'{r2_score.__name__}: {r2_score(y_test, skl_tree.predict(x_test))}')

r2_score: 0.6860847825591382


## Classification

In [13]:
x, y = make_classification(n_samples=2000, n_features=5, n_classes=4, n_clusters_per_class=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

ohe = OneHotEncoder(sparse=False)
y_train = ohe.fit_transform(y_train.reshape(-1, 1))
y_test = ohe.transform(y_test.reshape(-1, 1))

In [14]:
def softmax(z):
    '''
    Softmax function, z - matrix with shape [n_objects; n_classes]
    '''
    return np.exp(z) / np.sum(np.exp(z), axis=1).reshape(-1, 1)

In [15]:
class GradientBoostingClassifier:
    def __init__(self, lr=0.1, n_iter=100, **model_args):
        self.lr = lr
        self.n_iter = n_iter
        self.model = DecisionTreeRegressor(**model_args) # yes, we build regressor
        self.models = dict()
        self.n_classes = None
            
    def loss(self, y_true, a_pred):
        '''
        Compute cross-entropy loss, a_pred - matrix with shape [n_objects; n_classes]
        '''
        loss = 0
        for i, j in enumerate(y_true):
            loss += -np.log(a_pred[i][j])
        return loss
            
    def fit(self, x, y, test_set=None, echo=True):
        self.n_classes = y.shape[1]
        y_not_ohe = np.argmax(y, axis=1)
        
        # for each class
        approximation = np.zeros((x.shape[0], self.n_classes))
        for i in range(self.n_iter):
            # for every iteration we create n models for n classes
            self.models[i] = dict()
            
            if echo:
                print(f'LOSS {i+1}: {self.loss(y_not_ohe, softmax(approximation))}')
            
            # because we build models independently, we need to normalize their preds with softmax
            grad = -(y - softmax(approximation))
            
            preds = np.zeros(shape=approximation.shape)
            for class_ in range(self.n_classes):
                
                # in multiclass classification we need to build n models for n classes
                
                model = copy.deepcopy(self.model)
                grad_for_class = grad[:, class_]

                model.fit(x, grad_for_class) # fit model on residuals
                preds[:, class_] = model.predict(x)
                
                # save model for iteration and class
                self.models[i][class_] = model

            approximation -= self.lr * preds
            if test_set is not None:
                x_t, y_t = test_set
                y_t = np.argmax(y_t, axis=1)
                preds = self.predict(x_t)
                if echo:
                    print(f'TEST LOSS {i+1}: {self.loss(y_t, softmax(preds))}')

        return self
    
    def predict(self, x):
        approximation = np.zeros((x.shape[0], self.n_classes))
        for i in self.models.keys():
            preds = np.zeros(approximation.shape)
            for class_ in range(self.n_classes):
                preds[:, class_] = self.models[i][class_].predict(x)
            approximation -= self.lr * preds
        return softmax(approximation)
    
    def __repr__(self):
        return f'GradientBoostingClassifier(lr={self.lr}, n_iter={self.n_iter})'

In [16]:
boosting_clf = GradientBoostingClassifier(lr=0.1, n_iter=463, max_depth=4)

In [17]:
boosting_clf.fit(x_train, y_train)

LOSS 1: 2218.0709777917327
LOSS 2: 2120.070624268176
LOSS 3: 2026.971593466313
LOSS 4: 1938.8038272004626
LOSS 5: 1855.3734589248836
LOSS 6: 1776.234561580107
LOSS 7: 1701.5786548782773
LOSS 8: 1631.2732790211442
LOSS 9: 1565.1385359305268
LOSS 10: 1502.8443568536184
LOSS 11: 1444.4323162306125
LOSS 12: 1389.076569508724
LOSS 13: 1336.9876236122118
LOSS 14: 1288.1160258322657
LOSS 15: 1242.5571687222018
LOSS 16: 1199.369246284634
LOSS 17: 1157.883091347727
LOSS 18: 1118.5351949900833
LOSS 19: 1081.6573490618784
LOSS 20: 1047.0594576724295
LOSS 21: 1014.8633349968707
LOSS 22: 984.0608417199876
LOSS 23: 955.0097234544801
LOSS 24: 927.6306815545908
LOSS 25: 901.9091443512631
LOSS 26: 877.4600595007527
LOSS 27: 854.3908378621877
LOSS 28: 832.5504801541579
LOSS 29: 811.8315311190581
LOSS 30: 791.9356493462958
LOSS 31: 772.7360826074299
LOSS 32: 754.9316910821962
LOSS 33: 737.9658133381805
LOSS 34: 721.9021302563305
LOSS 35: 706.6346317803997
LOSS 36: 692.1238085622965
LOSS 37: 678.267345936

GradientBoostingClassifier(lr=0.1, n_iter=463)

In [18]:
test_preds = boosting_clf.predict(x_test)
test_preds

array([[0.13145008, 0.01047117, 0.84816969, 0.00990906],
       [0.00693707, 0.00642174, 0.00642956, 0.98021164],
       [0.00661955, 0.98092344, 0.00606113, 0.00639588],
       ...,
       [0.07470787, 0.01879986, 0.89008493, 0.01640733],
       [0.0063783 , 0.00653321, 0.00512742, 0.98196107],
       [0.98225453, 0.00543585, 0.00637363, 0.00593599]])

In [19]:
train_preds = boosting_clf.predict(x_train)
print(f'TRAIN acc: {accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_preds, axis=1))}')
print(f'TEST acc: {accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_preds, axis=1))}')

TRAIN acc: 0.9975
TEST acc: 0.945


# Binary classification

## Explanation based on a video, check it for details:
[Gradient Boost Part 4 (of 4): Classification Details](https://www.youtube.com/watch?v=StWY5QWMXCw&t=1810s)

In examples above we missed some details of boosting

First of all we need data and differentiable loss function

![](./images/gb_input.png)

Assume for classification we have log loss

![](./images/log_loss.png)

Fist step after that we need to make first approximation as a constant prediction

![](./images/gb_step1.png)

And then huge step 2, where we train model and build approximation

![](./images/gb_step2.png)

- (A) We compute gradient of a loss
- (B) We fit regression tree on gradient and create leaves
- (C) For each leaf if tree we change its output value to be the argmin of equation above
- (D) We update our approximation

In part C for loss used in GBDT for regression we will see, that no need to change tree outputs since it will be mean of values got to the leaf, but for log-loss and potentially for other losses it is needed

In [20]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [26]:
class GBDTBinary:
    def __init__(self, lr=0.1, n_iter=100, **model_kwargs):
        self.lr = lr
        self.n_iter = n_iter
        self.model = DecisionTreeRegressor(**model_kwargs)
        self.models = []
        self.mean_y = None
        
        for i in range(n_iter):
            self.models.append(copy.deepcopy(self.model))
            
    def fit(self, x, y):
        self.mean_y = y.mean()
        approximation = np.full((x.shape[0]), self.mean_y/(1-self.mean_y)).reshape(-1)
        # approximation = np.zeros((x.shape[0])).reshape(-1)
        
        for i, model in enumerate(self.models):

            # get probabilities from logits
            probs = sigmoid(approximation)

            # write log loss, take a look at it, be sure it reduces
            loss = -np.mean(y * np.log(probs) + (1 - y) * np.log(1-probs))
            print(f"LOSS {i}: {loss}")

            # gradient of loss with respect to logits, simplify it
            # grad = -y * (1 - probs) + (1 - y) * probs
            # grad = -y * y*probs + probs - y*probs
            grad = probs - y
            
            model.fit(x, grad) # fit model on residuals
            # here comes important part, you can see how it affects if you comment and run code without it
            # we need to change model outputs in leaves to (sum(grads_in_leaf) / (sum(probs_for_v_in_leaf*(1-probs_for_v_in_leaf))))
            # it comes from the part when we solve the quation
            leaves = model.tree_.apply(x.astype(np.float32))
            for leave in np.unique(leaves):
                indexes = np.where(leaves == leave)
                grads = grad[indexes]
                ps = probs[indexes]
                value = np.sum(grads) / np.sum(ps*(1-ps))
                model.tree_.value[leave] = value

            preds = model.predict(x)
            approximation -= self.lr * preds
        return self
    
    def predict(self, x):
        approximation = np.full((x.shape[0]), self.mean_y/(1-self.mean_y)).reshape(-1)
        # approximation = np.zeros((x.shape[0])).reshape(-1)
        for model in self.models:
            preds = model.predict(x)
            approximation -= self.lr * preds
        return sigmoid(approximation)
    
    def __repr__(self):
        return f'GradientBoostingClassifier(lr={self.lr}, n_iter={self.n_iter})'

In [75]:
x, y = make_classification(n_samples=15000, n_features=50, n_informative=10, n_redundant=2, n_repeated=3, shift=2.7, scale=5.1)

In [76]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.33)

In [77]:
gbdt_binary = GBDTBinary(lr=0.1, n_iter=1000, max_depth=3)

In [78]:
gbdt_binary.fit(x_train, y_train)

LOSS 0: 0.8129270043644506
LOSS 1: 0.750985465578975
LOSS 2: 0.7057954032021805
LOSS 3: 0.6650113681444174
LOSS 4: 0.6358135375088041
LOSS 5: 0.6086624392968454
LOSS 6: 0.5845033294356913
LOSS 7: 0.5628287424290369
LOSS 8: 0.5463224157122523
LOSS 9: 0.5279123541743334
LOSS 10: 0.5116335985398291
LOSS 11: 0.49697837250052057
LOSS 12: 0.48704256975934945
LOSS 13: 0.47567577247243964
LOSS 14: 0.46415257548179467
LOSS 15: 0.4521401597008696
LOSS 16: 0.44326909701384587
LOSS 17: 0.4346539587090187
LOSS 18: 0.4269089819794095
LOSS 19: 0.4192798027848657
LOSS 20: 0.4130718128828772
LOSS 21: 0.4056151731767842
LOSS 22: 0.39945111289011825
LOSS 23: 0.393146872029087
LOSS 24: 0.38815617515918743
LOSS 25: 0.3830179462167261
LOSS 26: 0.3783243453297911
LOSS 27: 0.3732748677080617
LOSS 28: 0.3677188475587334
LOSS 29: 0.36307463744613333
LOSS 30: 0.3585753764519507
LOSS 31: 0.3554832446144048
LOSS 32: 0.35237796581119585
LOSS 33: 0.34686742442907437
LOSS 34: 0.3425880246396712
LOSS 35: 0.34003995984

GradientBoostingClassifier(lr=0.1, n_iter=1000)

In [79]:
gbdt_binary.predict(x_val)

array([0.00257459, 0.99939806, 0.05021223, ..., 0.02199869, 0.70288902,
       0.09005021])

In [80]:
preds_train = gbdt_binary.predict(x_train)
preds_val = gbdt_binary.predict(x_val)

print(f'Train ROC AUC: {roc_auc_score(y_train, preds_train)}')
print(f'Val ROC AUC: {roc_auc_score(y_val, preds_val)}')

Train ROC AUC: 0.9999725927308318
Val ROC AUC: 0.9773145176473395


In [81]:
gbdt_sklearn = GradientBoostingClassifier(learning_rate=0.1, n_estimators=1000, max_depth=3, criterion='squared_error')

In [82]:
gbdt_sklearn.fit(x_train, y_train)

GradientBoostingClassifier(criterion='squared_error', n_estimators=1000)

In [83]:
preds_train_skl = gbdt_sklearn.predict(x_train)
preds_val_skl = gbdt_sklearn.predict(x_val)

print(f'Train ROC AUC: {roc_auc_score(y_train, preds_train_skl)}')
print(f'Val ROC AUC: {roc_auc_score(y_val, preds_val_skl)}')

Train ROC AUC: 0.9975199193814385
Val ROC AUC: 0.9264286930774908


In [55]:
with open('./style.css', 'r') as f:
    style = f.read()
HTML(style)